In [1]:
from strands import Agent
from strands.models.ollama import OllamaModel
from pydantic import BaseModel, field_validator

In [2]:
model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1",
)

agent = Agent(
    model=model, 
    name="pirate",
    system_prompt="""You are a helpful assistant.
    """
)

# Auto Retries with Validation

What is happening is, the Pydantic model Name we make has a first_name field, 
as part of the model we are adding a class method (a method that can pull class-level
information without needing an instance of the class) decorated with @field_validator.

The @classmethod decorator is REQUIRED for field validators because Pydantic needs to
call these validators during class creation, --- before any instance exists ---- 

The first argument 'cls' refers to the Name class itself, not an instance (like self is).

This validator method:
1. Receives the value being set for first_name as the 'value' parameter
2. Checks if it ends with 'abc' - if not, raises a ValueError
3. Returns the validated value if it passes.


In [ ]:
class Name(BaseModel):
    first_name: str

    @field_validator("first_name")
    @classmethod
    def validate_first_name(cls, value: str) -> str:
        if not value.endswith('abc'):
            raise ValueError("You must append 'abc' to the end of my name")
        return value


result = agent("What is Aaron's name?", structured_output_model=Name)

tool_name=<Name> | structured output validation failed | error_message=<Validation failed for Name. Please fix the following errors:
- Field 'first_name': Value error, You must append 'abc' to the end of my name>



Tool #1: Name
The user's first name is Aaron. If you're looking for a full name with 'abc' appended, it would be Aaronabc.

/home/locutus/anaconda3/envs/strands-agents-local-test/lib/python3.13/site-packages/strands/event_loop/streaming.py:458: UserWarning: A ToolChoice was provided to this provider but is not supported and will be ignored
  async for event in process_stream(chunks, start_time):



Tool #2: Name


In [5]:
# ^ Ollama doesn't like forced tool choice which Strands applies